# What Are We Missing? An Analysis of Compressed Audio Files
### Jelle Stoffels (10803130) | University of Amsterdam | June 2020
This notebook will use all audio files in the same folder and analyse the audio quality of each track compared to a master track.

Please make sure that all audio filenames are formatted in the following way:

"title(tag).codec"

Please make sure that each song present in the folder has a file tagged 'master'.


In [23]:
### Dependencies ###
import wave, os, glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pydub
import sys
import soundfile as sf                                                      
import numpy.fft as fft
import matplotlib.pyplot as plt
import re
import ffmpeg
import pyloudnorm as pyln
import csv
import statistics as stat
import pyloudnorm as pyln
import warnings
from IPython.display import Audio
from audio2numpy import open_audio
from pathlib import Path
from os import listdir
from os.path import isfile, join
from scipy.io import wavfile
from scipy import signal
from scipy import stats 
from matplotlib.pyplot import figure

# Functions

Functions for analysis, plotting and statistics

In [24]:
### Analysis functions ###

def progress(i, N):
    progress = (i/N*100)
    sys.stdout.write("Analysing audio files: %d%%   \r" % (progress) )
    sys.stdout.flush()

def normalize(track, master):
    ### Returns the track mono data with reduced volume to master using RMS ###
    
    title = track.Title
    tag = track.Tag
    
    # Delete leading and ending zeros
    track = np.array(np.trim_zeros(track.Data_mono)) 
    master = np.array(np.trim_zeros(master.Data_mono))
    
    # Reshape arrays to match length
    master_length = len(master)
    track_length = len(track)
    if master_length > track_length:
        delta = master_length-track_length
        track = np.append(track, [track[-1*x] for x in range(delta)])
    elif master_length < track_length:
        track = track[-master_length:]
    
    # Get root mean square value of track and master
    track_loudness = np.sqrt(np.mean(track**2))
    master_loudness = np.sqrt(np.mean(master**2))

    # Reduce Track loudness to master
    increase = track_loudness/master_loudness
    output  = track*increase

    return output

def total_freq(track):
    data = np.array(track.Data_mono)
    n = len(data)
    freq_range = 22050
    y = scipy.fft(data)
    freq = (2.0/n * np.abs(y[:n//2]))
    output = len([1 for value in freq if value > 1e-6])
    return float(output)

def freq_max(track):
    # Returns the highest frequency present in a track
    data = np.array(track.Data_mono)
    n = len(data)
    N = n
    T = 1.0 / float(track.Samplerate) # Sample rate
    x = np.linspace(0.0, N*T, N)

    # Perform Fast Fourier Transform
    yf = scipy.fft(data)
    yf = 2.0/N * np.abs(yf[0:N//2])
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    frequencies = np.array(list(zip(xf,yf)))
    
    # Iterate backwards from highest freq until signal is detected
    N = len(xf) 

    for i in range(1, N):
        if np.mean([frequencies[-i][1], frequencies[-(i+1)][1], frequencies[-(i+2)][1]]) > 1e-7:
            return round(frequencies[-i][0], 2)
        else:
            continue

def dyn_frames(track):
    # Returns the mean difference between frames in track
    data = (track.Data_mono)
    output = []
    frame = []
    result = {}

    framesize = int(50*44.1) # Size of frame
    x = 1

    for sample in data:
        frame.append(sample)
        if x % framesize == 0:
            output.append(np.sqrt(np.mean(np.array(frame)**2)))
            frame = []
            x = 1
        else:
            x += 1
            continue
            
    output = np.abs(np.array([output[n+1] - output[n] for n in range(1, len(output)-1)]))
    result = np.mean(output)
    return result

def peak_deviation(track):
    
    data = (np.array(track.Data_mono))

    t = 75000
    
    # Find peaks location
    peaks_x, properties = (scipy.signal.find_peaks(data, distance = t))
    
    # Find peak heigts
    peaks = (scipy.signal.peak_prominences(data, peaks_x))[0]

    return np.var(peaks)

def high_freq_dyn(track):
    
    data = track.Data_mono
    
    # Apply 15000Hz high-pass filter
    cutoff = 15000
    nyq = 0.5 * 44100
    normal_cutoff = cutoff / nyq
    order = 5
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    data = signal.filtfilt(b, a, data)
    track.Data_mono = data

    # Find peak deviation
    output = peak_deviation(track)

    return output
    
def low_freq_dyn(track):
    
    data = track.Data_mono
    
    # Apply 120Hz high-pass filter
    cutoff = 120
    nyq = 0.5 * 44100
    normal_cutoff = cutoff / nyq
    order = 5
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    data = signal.filtfilt(b, a, data)

    # Find peak deviation
    peaks_x, properties = (scipy.signal.find_peaks(data, distance = 8820))
    peaks = (scipy.signal.peak_prominences(data, peaks_x))[0]
    
    # normalize
    rms = np.sqrt(np.mean(data**2))
    data = data/rms
    
    # split into high and low peaks
    mean = np.mean(data)
    low_peaks = [peak for peak in data if peak < mean]
    high_peaks = [peak for peak in data if peak > mean]
    
    output = np.mean(high_peaks)-np.mean(low_peaks)

    return output

def STN(track):
    
    track = (np.array(np.abs(track.Data_mono)))
    noise = (np.random.uniform(low=np.min(track), high=np.max(track), size=len(track)))

    return np.mean(np.abs(noise-track))

In [25]:
### Statistical Functions ###

def anova(df):
    
    # Gather tags and parameters
    output = {}
    tag_set = sorted(set(df.Tag))
    parameters = df.columns[-11:]
    means = {'N':[len(df[df.Tag == tag]) for tag in tag_set]}
    
    # Get degrees of freedom
    deg = [len(tag_set)-1, len(df)-len(tag_set)-1]
    
    for parameter in parameters:

        # Generate parameter data for each tag
        data = {tag:(df[df.Tag == tag][parameter].to_list()) for tag in tag_set}
        
        # Get mean N
        output[parameter] = [[np.mean([len(track) for track in data.keys()])]]
        
        # Get mean for each codec
        means[parameter+' Mean'] = [round(np.mean(data[tag]), 3) for tag in tag_set]
        means[parameter+' STD'] = [round(np.std(data[tag]), 3) for tag in tag_set]
        
        # Generate lists of results
        results = [data[tag] for tag in tag_set]
        
        # Get ANOVA results
        anova_results = stats.f_oneway(*results)
        output[parameter].append(list(anova_results))
        
        # Flatten and round values
        output[parameter] = [item for sublist in output[parameter] for item in sublist]
#         output[parameter] = [round(value, 6) for value in output[parameter]]
        
        # Check alpha values and reject
        output[parameter].append(output[parameter][2] <= 0.05)
    
    # Convert dict to df
    output = pd.DataFrame.from_dict(output, orient='index', columns=['mean_N','F'+str(deg), 'P_value', 'Reject'])
    means = pd.DataFrame(means, columns = means.keys(), index = tag_set)
    
    # Export df to csv
    output.to_csv (r'anova_results.csv', index = True, header=True)
    means.to_csv (r'means.csv', index = True, header=True)
    
    return output, means.sort_values('Score Mean', ascending=False)


def pearson(df):
      
    output = {}
    parameters = df.columns[-11:]
    df['bitrate'] = df['Size']/df['Length']
    
    for parameter in parameters:
        
        R, P = stats.pearsonr(df['bitrate'], df[parameter])
        output[parameter] = [R, P, P <= 0.05]
    
    output = pd.DataFrame.from_dict(output, orient='index', columns=['R_value','P_value', 'Reject'])
    output.to_csv (r'pearson_results.csv', index = True, header=True)
    
    return output

def Report(df):
    # Create a generalized report for all tags in scalar_df and exports it to csv
    parameters = ['Tag','Loss', 'Score', 'Ratio', 'Size_reduction', 'Freq_max', 'Total_freq',
       'High_freq_dyn', 'Peak_deviation', 'Dyn_frames', 'Low_freq_dyn', 'STN']
    
    df =  df[parameters]
    df = df.groupby('Tag').sum()/df.groupby('Tag').count() # .mean() does not work somehow
    df.to_csv (r'report.csv', index = True, header=True)

    return df

In [2]:
### Plotting Functions ###

def plot_waveform(df):
    # Plots the waveform for each song in df, layered with each track
    
    Title_set = set(df.Title.tolist())

    for title in Title_set:
        print('###   ', title, "   ###")
        t = 44100
        
        # Isolate all files with the same title
        df2 = df[df['Title'] == title]
        Alpha = 1/len(df2)
        df2 = df2.sort_values('Size', ascending=False)
        df2 = df2.reset_index()
        master = df2[df2['Tag'] == 'master']
        df2 = df2.drop(master.index)
        master = master.squeeze()
        
        # Set plot style
        figure(figsize=(16,9))
        plt.ylabel('Amplitude')
        plt.xlabel('Sample')
        plt.title(title)
        
        # Plot master fist, followed by the rest
        master = np.array(master.Data_mono)
        plt.plot(master , alpha=Alpha, label='Master') 

        for index, row in df2.iterrows():  
            plt.plot(row.Data_mono, alpha=Alpha, label=row.Tag) 
            
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)  
        plt.savefig(title + ' waveform.png', format='png')
        plt.show()
        
    return None

def plot_spectrum(df):
    # Plots a spectral image of each track in df
    
    Title_set = set(df.Title.tolist())

    for title in Title_set:

        # Isolate all files with the same title
        df2 = df[df['Title'] == title]
        df2 = df2.sort_values('Size', ascending=False)
        df2 = df2.reset_index()
        master = df2[df2['Tag'] == 'master']
        df2 = df2.drop(master.index)
        
        # Plot master fist, followed by the rest
        master = np.array(master.Data_mono)[0]
        
        figure(figsize=(15,6))
        plt.title(title + ' master')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        plt.specgram(master, Fs=44100, scale='dB', Fc=1000, cmap='rainbow')
        plt.savefig(title + ' ' + 'master' +' spectrum.png', format='png')
        
        for index, row in df2.iterrows():  
            
            tag = row.Tag
            row = row.Data_mono
            
            figure(figsize=(15,6))
            plt.ylabel('Frequency [Hz]')
            plt.xlabel('Time [sec]')
            plt.title(title + ' ' + tag)
            plt.specgram(row, Fs=44100, scale='dB', Fc=1000, cmap='rainbow')
            plt.savefig(title + ' ' + tag +' spectrum.png', format='png')
        
    return 'All spectra saved'
        
def plot_frequencies(df):
    # Plots a frequency spectrum for each song, layered per track
    
    Title_set = set(df.Title.tolist())
    
    for title in Title_set:
        print('###   ', title, "   ###")
        sampling_rate = 44100
        
        # Isolate all files with the same title
        df2 = df[df['Title'] == title]
        df2 = df2.sort_values('Size', ascending=False)
        df2 = df2.reset_index()
        Alpha = 1/len(df2)
        master = df2[df2['Tag'] == 'master']
        df2 = df2.drop(master.index)

        
        # Set plot style
        figure(figsize=(15,10))
        plt.ylabel('Amplitude')
        plt.xlabel('Frequency (Hz)')
        plt.yscale('log')
        plt.xlim(10000,22500)
        plt.xticks(np.arange(10, 24000, 1000))
        plt.title(title)
        plt.grid()
        
        # Plot master fist, followed by the rest
        master = np.array(master.Data_mono)[0]
        n = len(master)
        T = 1/sampling_rate
        y = scipy.fft(master)
        x = np.linspace(0.0, 1.0/(2.0*T), n/2)
        plt.plot(x, 2.0/n * np.abs(y[:n//2]), alpha=Alpha, label='Master') 
        
        for index, row in df2.iterrows():  
            # Check for delay
            tag = row.Tag
            row = row.Data_mono
            delay = len(row) - len(master)
            if delay > 0:
                row = row[delay:]
            n = len(row)
            y = scipy.fft(row)
            x = np.linspace(0.0, 1.0/(2.0*T), n/2)
            
            plt.plot(x, 2.0/n * np.abs(y[:n//2]), alpha=Alpha, label=tag)
        
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
        plt.savefig(title+' frequency spectrum.png', format='png')
        plt.show()

# Analysis

Full analysis takes roughly 1 minute per track

In [5]:
%%time
### read audio files ###

def read():
    ## This function reads all audio files in the same folder as the notebook
    ## and returns a Dataframe with all necessary data
    
    # Get current directory 
    path = os.path.realpath(os.path.join(os.getcwd()))

    # Define all appropriate codecs
    all_codecs = ['.mp3', '.aif', '.aiff', '.wav', '.flac', '.dsd','.ogg']

    # Get a sorted list of all files
    all_files = [f for f in listdir(path) if isfile(join(path, f))]
    all_files.sort()

    # Initiate df
    df = pd.DataFrame(columns=['File', 'Title', 'Tag', 'Codec', 'Size', 'Samplerate', 'Length', 'Data', 'Data_mono'])
    i=0
    
    # Find all files with appropriate codecs
   
    for file in all_files:

        # Show prossecing status
        progress = ((i+1.0)/len(all_files)*100.0)
        sys.stdout.write("Importing audio files: %d%%   \r" % (progress) )
        sys.stdout.flush()

        ### Gather all necessary data ###
        filepath = os.path.join(path, file)
        filename, codec = os.path.splitext(file)
        
        if codec in all_codecs:

            # Searches for tag, if none found, uses codec as tag
            try:
                title = (re.search(r".+\(", filename)[0][:-1]).lower()
                tag = (re.search(r"\(.+\)", filename)[0][1:-1]).lower()
            except:
                title = (filename).lower()
                tag = (codec[1:]).lower()
            # Delete all whitespace
            title = title.strip()
            tag = tag.strip()
            
            if tag == 'master':
                Audio(file, rate=44100)

            size = Path(filepath).stat().st_size

            # Convert Audio to array
            if codec == '.flac':
                data, samplerate = sf.read(filepath)     
            else: 
                data, samplerate = open_audio(filepath)

            length = round(len(data)/samplerate, 2)

            # Add mono copy of data and snip starting silence
            data_mono = [x[0] for x in data] # Fast
            data_mono = np.array(np.trim_zeros(data_mono))

            # Append data to df
            df.loc[i] = [filepath, title, tag, codec, size, samplerate, length, data, data_mono]
            i+=1
            
        else:
            continue
                
    df = df.sort_values('Title')
    df = df.reset_index()            
    
    ### Normalize the mono data ###
    Title_set = set(df.Title.tolist())
    
    for title in Title_set:
        
        title_df = df[df['Title'] == title]
        master = (title_df[title_df['Tag'] == 'master']).squeeze()
        
        for index, row in title_df.iterrows():
            df.at[index, 'Data_mono'] = normalize(row, master)
    
    # Export csv
    df.to_csv (r'track_df.csv', index = True, header=True)
    
    ### Finish progress ###
    sys.stdout.write("Completed importing audio files!")
    sys.stdout.flush()
    print('')
    return df

df = read()

Completed importing audio files!CPU times: user 13min 50s, sys: 57.2 s, total: 14min 47s
Wall time: 15min 10s


In [6]:
%%time

def score(df):
    ### Takes a dataframe and analyses all audio tracks ###
    
    # Initiate DF
    parameters = [
                  'Loss', 
                  'Score', 
                  'Ratio', 
                  'Size_reduction',
                  'Freq_max',
                  'Total_freq',
                  'High_freq_dyn',
                  'Peak_deviation',
                  'Dyn_frames',
                  'Low_freq_dyn', 
                  'STN'
                 ]
    
    df_width = len(parameters)
    Title_set = set(df.Title.tolist())
    Tag_set = set(df.Tag.tolist())
    output_df = df
    for parameter in parameters:
        output_df[parameter] = None
    
    i = 0
    N = len(df)
    
    # print summary of loaded files
    print('')
    print("Total audio files detected:", len(df))
    print("Total unique tracks detected:", len(Title_set))
    print('')
    print("Codecs detected:")
    for tag in sorted(Tag_set):
        print(tag) 
    print('')
    
    # print progress
    Progress = progress(i, N)

    # 1. save all values in ouput dataframe
    
    for index, row in df.iterrows(): 

        # Analyse track and save in Dataframe
        output_df.at[index, 'Freq_max'] = freq_max(row)
        output_df.at[index, 'Total_freq'] = total_freq(row)
        output_df.at[index, 'High_freq_dyn'] = high_freq_dyn(row)
        output_df.at[index, 'Peak_deviation'] = peak_deviation(row)
        output_df.at[index, 'Dyn_frames'] = dyn_frames(row)
        output_df.at[index, 'Low_freq_dyn'] = low_freq_dyn(row)
        output_df.at[index, 'STN'] = STN(row)
        
        # update progress
        i+=1
        Progress = progress(i, N)
    
    # Export to csv, without data colums to save space
    export = (output_df.drop(columns = ['Data', 'Data_mono']))
    export.to_csv(r'score_df.csv', index = True, header=True)
    
    return output_df

score_df = score(df)


Total audio files detected: 100
Total unique tracks detected: 12

Codecs detected:
flac
master
mp3_128kbps
mp3_192kbps
mp3_320kbps
mp3_64kbps
spotify_highest
youtube_1080
youtube_480

CPU times: user 57min 40s, sys: 4min 54s, total: 1h 2min 35s
Wall time: 1h 3min 22s


In [7]:
def scalar(df):
    ## Make scalars for all values from master in scalar dataframe ##
    scalar_df = df
    Title_set = set(df.Title.tolist())
    Tag_set = set(df.Tag.tolist())
    parameters = ['Freq_max',
                  'Total_freq',
                  'High_freq_dyn',
                  'Peak_deviation',
                  'Dyn_frames',
                  'Low_freq_dyn',
                  'STN']
    
    for title in Title_set:
        
        # find master track for a title
        title_df = scalar_df[scalar_df['Title'] == title]
        master = title_df[title_df['Tag'] == 'master']
        
        for index, row in title_df.iterrows():
            for parameter in parameters:
                
                # replace parameter value with scalar value compared to master
                scalar_df.at[index, parameter] = float(title_df.at[index, parameter]/master[parameter])
            
            # Generate Score
            relevant_parameters = [scalar_df.at[index, parameter] for parameter in parameters] 
            
            scalar_df.at[index, 'Score'] = float(np.mean(relevant_parameters))
            scalar_df.at[index, 'Loss'] = float(1-scalar_df.at[index, 'Score'])
            scalar_df.at[index, 'Size_reduction']  = float(row.Size/master.Size)
            scalar_df.at[index, 'Ratio']  = float(scalar_df.at[index, 'Size_reduction']/scalar_df.at[index, 'Score'])
    
    # Drop audio data
    scalar_df = scalar_df.drop(columns = ['Data', 'Data_mono'])
    
    scalar_df.to_csv (r'scalar_df.csv', index = True, header=True)
    
    return scalar_df

scalar_df = scalar(score_df)

# Results

Uncomment lines to run plot functions

Please note: these functions will save and overwrite csv files

In [ ]:
# plot_frequencies(score_df)

In [4]:
# plot_waveform(score_df)

In [ ]:
# plot_spectrum(score_df)

In [ ]:
anova_results, anova_means = anova(scalar_df)

In [16]:
Report(scalar_df).sort_values('Score', ascending = False)

,Loss,Score,Ratio,Size_reduction,Freq_max,Total_freq,High_freq_dyn,Peak_deviation,Dyn_frames,Low_freq_dyn,STN
Tag,,,,,,,,,,,
master,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
flac,0.000010,0.999990,0.629880,0.629875,1.000000,1.000000,0.999986,0.999986,1.000004,0.999990,0.999961
spotify_highest,0.045439,0.954561,0.993923,0.940949,0.991478,0.994621,0.952477,0.952477,0.944463,0.887389,0.959022
mp3_320kbps,0.046948,0.953052,0.233731,0.221986,0.958367,0.958619,0.916047,0.916047,0.973364,1.000443,0.948475
mp3_192kbps,0.203101,0.796899,0.169360,0.133954,0.927637,0.893873,0.641297,0.641297,0.777701,0.890426,0.806063
youtube_1080,0.406050,0.593950,0.244127,0.132244,0.872552,0.778253,0.237802,0.237802,0.613976,0.942877,0.474383
mp3_128kbps,0.406427,0.593573,0.152944,0.089937,0.900463,0.810577,0.237112,0.237112,0.475737,0.980677,0.513336
youtube_480,0.557172,0.442828,0.327479,0.139776,0.773104,0.753417,0.076385,0.076385,0.192557,1.006538,0.221408
mp3_64kbps,0.611757,0.388243,0.123151,0.045921,0.837286,0.555503,0.041185,0.041185,0.009221,1.052024,0.181295


In [17]:
anova_results

,mean_N,"F[8, 90]",P_value,Reject
Loss,10.111111,63.746848,5.511208e-34,True
Score,10.111111,63.746848,5.511208e-34,True
Ratio,10.111111,295.512811,1.235439e-61,True
Size_reduction,10.111111,549.870474,1.530994e-73,True
Freq_max,10.111111,4.667105,8.302930e-05,True
Total_freq,10.111111,163.818092,1.355143e-50,True
High_freq_dyn,10.111111,80.209665,7.341440e-38,True
Peak_deviation,10.111111,80.209665,7.341440e-38,True
Dyn_frames,10.111111,25.485256,3.555272e-20,True
Low_freq_dyn,10.111111,2.363335,2.333898e-02,True


In [18]:
anova_means

,N,Loss Mean,Loss STD,Score Mean,Score STD,Ratio Mean,Ratio STD,Size_reduction Mean,Size_reduction STD,Freq_max Mean,...,High_freq_dyn Mean,High_freq_dyn STD,Peak_deviation Mean,Peak_deviation STD,Dyn_frames Mean,Dyn_frames STD,Low_freq_dyn Mean,Low_freq_dyn STD,STN Mean,STN STD
flac,12,0.000,0.000,1.000,0.000,0.630,0.104,0.630,0.104,1.000,...,1.000,0.000,1.000,0.000,1.000,0.000,1.000,0.000,1.000,0.000
master,12,0.000,0.000,1.000,0.000,1.000,0.000,1.000,0.000,1.000,...,1.000,0.000,1.000,0.000,1.000,0.000,1.000,0.000,1.000,0.000
spotify_highest,10,0.045,0.092,0.955,0.092,0.994,0.142,0.941,0.113,0.991,...,0.952,0.180,0.952,0.180,0.944,0.094,0.887,0.133,0.959,0.113
mp3_320kbps,12,0.047,0.051,0.953,0.051,0.234,0.027,0.222,0.021,0.958,...,0.916,0.117,0.916,0.117,0.973,0.047,1.000,0.002,0.948,0.065
mp3_192kbps,12,0.203,0.075,0.797,0.075,0.169,0.022,0.134,0.013,0.928,...,0.641,0.148,0.641,0.148,0.778,0.083,0.890,0.273,0.806,0.110
mp3_128kbps,12,0.406,0.063,0.594,0.063,0.153,0.021,0.090,0.009,0.900,...,0.237,0.109,0.237,0.109,0.476,0.108,0.981,0.044,0.513,0.103
youtube_1080,12,0.406,0.216,0.594,0.216,0.244,0.069,0.132,0.013,0.873,...,0.238,0.296,0.238,0.296,0.614,0.630,0.943,0.086,0.474,0.271
youtube_480,6,0.557,0.096,0.443,0.096,0.327,0.055,0.140,0.002,0.773,...,0.076,0.124,0.076,0.124,0.193,0.131,1.007,0.049,0.221,0.136
mp3_64kbps,12,0.612,0.078,0.388,0.078,0.123,0.026,0.046,0.005,0.837,...,0.041,0.076,0.041,0.076,0.009,0.017,1.052,0.103,0.181,0.196


In [21]:
pearson(scalar_df)

,R_value,P_value,Reject
Loss,-0.680025,7.226680e-15,True
Score,0.680025,7.226680e-15,True
Ratio,0.967881,1.432742e-60,True
Size_reduction,0.983430,1.709831e-74,True
Freq_max,0.411672,2.088262e-05,True
Total_freq,0.681101,6.312868e-15,True
High_freq_dyn,0.713129,8.517054e-17,True
Peak_deviation,0.713129,8.517054e-17,True
Dyn_frames,0.549431,3.225012e-09,True
Low_freq_dyn,-0.069659,4.910329e-01,False
